In [1]:
import os
import sys
import re

import pandas as pd
import numpy as np

sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")

import datetime
from utils import get_path, DATETIME_FORMAT

from universal_connection import UniversalConnection, DBType

import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
from auto_width7 import auto_columns_width

In [3]:
sql = """
with temp as (
select 	a.patient_id,
		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
		p_patient.sex, 
		p_patient.birthday patient_birthday,
		p_patient.phone patient_phone,
		p_patient.email,
		case
  		when "source"='SCHEDULED' and specialty_id is not null then (
			select trim(name)
			from specialty s
			where s.id = a.specialty_id
		)
	    else (
			select trim(s.name)
			from specialty s
			inner join doctor_specialty ds
			   on ds.specialty_id = s.id
			inner join doctor_specialty_treatment dst
			   on dst.doctor_specialty_id = ds.id
			where dst.treatment_id = a.treatment_id
			limit 1 -- из-за одной криво заведенной записи.
		) end as specialty,
		a.report_comment,
		date(a.finished at time zone 'UTC') finished,
		case when pc.product_id in (
		            		select id from product
		            		where full_name like '%ВЭБ%') then 'VEBMED'
		        else 'ПРОЧИЕ'
		end promo_type
from appointment a
left join patient on a.patient_id = patient.id 
left join person p_patient on patient.person_id = p_patient.id
--
left join promotion p
	      on a.promotion_id = p.id
	    left join product_condition pc
	      on pc.id = p.product_condition_id
--
where a.good
      and not (
      	a.patient_id in (
	      	select patient_id
	      	from patient_categories pcat
	      	inner join reference rf
	      	   on rf.id=pcat.reference_id
	      	where rf.code='TEST'
	      	)
	    or lower(a.report_comment)='тест'
      	)
      	and a.finished at time zone 'UTC' > '2020-03-30'
)
select * from temp
where promo_type = 'VEBMED'

"""

In [4]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
df_sql = connection.query(sql)
connection.close()

2020-06-18 18:07:51 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-06-18 18:07:51 creating ssh tunnel to 172.16.100.19 as root...
2020-06-18 18:07:51 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 43855
2020-06-18 18:07:51 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-06-18 18:07:52 @telemed query:
                    with temp as (
				    select 	a.patient_id,
				    		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				    		p_patient.sex, 
				    		p_patient.birthday patient_birthday,
				    		p_patient.phone patient_phone,
				    		p_patient.email,
				    		case
				      		when "source"='SCHEDULED' and specialty_id is not null then (
				    			select trim(name)
				    			from specialty s
				    			where s.id = a.specialty_id
				    

In [5]:
int_words = []
file_name = 'cardio.txt'

In [6]:
a = open(file_name)
for line in a:
    #print(line)
    int_words.append(line.strip())
a.close()

In [7]:
int_words = list(set([morph.parse(x)[0].normal_form for x in int_words]))

In [8]:
int_words

['кардиолог',
 'артериальный',
 'ожирение',
 'давление',
 'сердце',
 'сосуд',
 'холестерин',
 'диета']

In [9]:
stop_words = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()

In [10]:
d_int_words = dict()

In [11]:
df_sql['parsed_comment'] = np.nan

for i in range(df_sql.shape[0]):
    temp = re.findall(r"[0123456789А-Яа-я-\\\/ё]+", df_sql['report_comment'][i])
    temp = [morph.parse(x.lower())[0].normal_form for x in temp if not x in stop_words]
    temp = [x for x in temp if len(x) > 2]
    
    if any(word in temp for word in int_words):
        
        df_sql['parsed_comment'][i] = ' '.join(temp)

        for word in temp:
            d_int_words[word] = d_int_words.get(word, 0) + 1

In [12]:
df_int_words = pd.DataFrame.from_dict(d_int_words, orient='index').sort_index().reset_index().rename(columns={'index':'word', 0:'amount'})

In [13]:
df_int_words = df_int_words.sort_values(by=['amount'], ascending=False).reset_index(drop=True)

In [14]:
df_int_words

,word,amount
0,рекомендовать,3291
1,диагностический,3122
2,диета,2922
3,ада,2687
4,узи,2108
...,...,...
17851,всязывать,1
17852,всякий,1
17853,перегревать,1
17854,втереть,1


In [15]:
df_sql[~df_sql['parsed_comment'].isna()]

,patient_id,patient_fio,sex,patient_birthday,patient_phone,email,specialty,report_comment,finished,promo_type,parsed_comment
108,3041391,Панкратова лариса владимировна,FEMALE,1969-10-13,79106303853,chaika8840@gmail.com,Гастроэнтеролог,"Жалобы на режущие боли по всему животу, возник...",2020-04-15,VEBMED,жалоба режущий живот возникать пупок возникать...
150,3501008,Калимуллина Ольга Николаевна,FEMALE,1978-11-08,79274466348,kalimullina_olga78@mail.ru,Эндокринолог,Пациентка обратилась с жалобами на набор массы...,2020-04-20,VEBMED,пациентка набор килограмм ранее наблюдаться пр...
201,3032048,Левчук Владимир Владимирович,MALE,2016-07-06,79125410414,_larina_vera_@mail.ru,Оториноларинголог,"Учитывая результаты обследования, приведенные...",2020-04-02,VEBMED,учитывать привести атопический рыба пшеница яй...
221,3032914,ЖИЛЕВИЧ Римма Ибрагимовна,FEMALE,1964-01-16,79119795909,rimma1105@rambler.ru,Эндокринолог,"Консультация проведена заочно по поводу сына, ...",2020-04-06,VEBMED,консультация заочно сын жил геннадий владимир ...
269,51300,Зайцева Нелли Владимировна,FEMALE,1973-01-03,79036308077,zaytseva_nv@inbox.ru,Педиатр,Учитывая жалобы и гипертонический криз в анамн...,2020-04-05,VEBMED,учитывать гипертонический криз возможно экг иш...
...,...,...,...,...,...,...,...,...,...,...,...
84206,4226082,Перепелица Екатерина Валерьевна,FEMALE,1988-07-14,79250454697,savelevaroga@yandex.ru,Невролог,Диагностическая гипотеза: Люмбаго с ишиасом.(...,2020-06-18,VEBMED,диагностический люмбаго ишиас мрт консультация...
84230,4088282,Шпак Екатерина Сергеевна,FEMALE,1989-05-25,79789627390,None,Педиатр,Ребенок 5 лет. Жалобы на обнаружение фосфатов ...,2020-06-18,VEBMED,ребёнок жалоба обнаружение фосфат моча рекомен...
84236,3764291,Зайченко Никита Анатольевич,MALE,1988-05-24,79152734571,None,Эндокринолог,Пациенту пройти консультацию эндокринолога пос...,2020-06-18,VEBMED,пациент посоветовать ревматолог эндокринный сф...
84245,4622929,Курашова Дарья Андреевна,FEMALE,2010-06-08,79036111204,None,Оториноларинголог,"Описанные симптомы, позволяют предположить , А...",2020-06-18,VEBMED,описать позволять аллергический коньюнктивит п...


In [16]:
df_patients = df_sql[~df_sql['parsed_comment'].isna()]

In [17]:
df_patients.groupby(['patient_fio', 'sex', 'patient_birthday', 'patient_phone', 'email'])[['patient_id']].count().reset_index().rename(columns={'patient_id':'consultations'})

,patient_fio,sex,patient_birthday,patient_phone,email,consultations
0,. Фари,FEMALE,1998-12-22,77711186598,princeska_98@inbox.ru,2
1,Borisov Roman,MALE,1967-07-13,79165314298,r.borisov@yahoo.com,1
2,Borsuk William,MALE,2019-10-28,41794206897,we@alex.bio,1
3,Butorina Irina Aleksandrovna,FEMALE,1965-05-22,79377752054,birena1@mail.ru,1
4,Cучкова Ирина Михайловна,FEMALE,1972-04-10,79600462199,,2
...,...,...,...,...,...,...
1820,швыдко иляна Александрровна,FEMALE,1978-08-31,79522833551,shvidko.ilana@yandex.ru,2
1821,шумилова диана ****,FEMALE,1995-06-22,79518574345,diana.shumilova.95@bk.ru,1
1822,югова наталья сергеевна,FEMALE,1961-05-16,79525202989,,1
1823,якимова Карина Витальевна,FEMALE,1986-05-17,79618915758,elyapw@mail.ru,1


In [18]:
df_fio_phone_mail = df_patients.groupby(['patient_fio', 'sex', 'patient_birthday', 'patient_phone', 'email'])[['patient_id']].count().reset_index().rename(columns={'patient_id':'consultations'})

In [19]:
global writer
writer = None

writer = auto_columns_width('nozoology-cardio.xlsx', df_int_words, 'words', writer)
writer = auto_columns_width('nozoology-cardio.xlsx', df_patients, 'consultations', writer)
writer = auto_columns_width('nozoology-cardio.xlsx', df_fio_phone_mail, 'patients', writer)
   

writer.save()

0 word 51.0
1 amount 11.9
0 patient_id 17.0
1 patient_fio 51.0
2 sex 11.9
3 patient_birthday 27.2
4 patient_phone 22.099999999999998
5 email 51.0
6 specialty 51.0
7 report_comment 51.0
8 finished 17.0
9 promo_type 17.0
10 parsed_comment 51.0
0 patient_fio 51.0
1 sex 11.9
2 patient_birthday 27.2
3 patient_phone 22.099999999999998
4 email 51.0
5 consultations 22.099999999999998
